<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20221211.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* Customer Analysis
* Iterators

In [2]:
import pandas as pd
import numpy as np
import plotnine as gg
from plotnine import ggplot, aes
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import datetime, date, time, timezone, timedelta 

import os
import re

from functools import reduce
from collections import defaultdict

# Customer ANalysis

In [4]:
os.listdir()

['.config',
 'scanner_data.csv',
 'online_retail_II.xlsx',
 'sales.csv',
 'sales_vs_discount.csv',
 '1_order_payments_dataset.csv',
 '1_orders_dataset.csv',
 'mock_sales_data.csv',
 '1_customers_dataset.csv',
 'groceries.csv',
 'test.csv',
 'mall_customers.csv',
 'online_retail.xlsx',
 'ecommerce-data.csv',
 'sample-sales-data.csv',
 'KPMG_VI_New_raw_data_update_final.xlsx',
 'match_result_test.csv',
 'sales_data_with_stores.csv',
 'bz_data_analyis.csv',
 'KPMG_VI_New_raw_data_update_final.csv',
 'sample_data']

In [6]:
df = pd.read_csv('KPMG_VI_New_raw_data_update_final.csv')
df['transaction_date'] = pd.to_datetime(df['transaction_date'], infer_datetime_format= True)
df['date'] = df['transaction_date'].to_numpy().astype('datetime64[D]')
df['week'] = df['date'] - df['date'].dt.weekday.astype('timedelta64[D]')
df = df[df['order_status'] == 'Approved']
df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,date,week
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0,2017-02-25,2017-02-20
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0,2017-05-21,2017-05-15
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0,2017-10-16,2017-10-16
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0,2017-08-31,2017-08-28
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0,2017-10-01,2017-09-25


In [7]:
cnt = df.groupby(['customer_id', 'week'])['list_price'].agg({'count','sum'}).reset_index()
cnt.columns = ['id','week','freq', 'rev']
cnt.head()

,id,week,freq,rev
0,1,2017-01-02,1,360.40
1,1,2017-02-20,1,71.49
2,1,2017-03-27,2,1963.56
3,1,2017-04-03,1,1577.53
4,1,2017-05-08,1,1720.70


In [10]:
info_f = pd.pivot_table(cnt, index = 'id', columns = 'week', values = 'freq').reset_index()
info_f = pd.melt(info_f, id_vars = 'id', var_name = 'week', value_name = 'freq').sort_values(['id','week'],ascending = [1,0])
info_f = info_f.groupby(['id']).head(3)
info_f = info_f.groupby(['id'])['freq'].sum().reset_index()
info_f.head()

,id,freq
0,1,2.0
1,2,0.0
2,3,0.0
3,4,0.0
4,5,1.0


In [16]:
info_m = pd.pivot_table(cnt, index = 'id', columns = 'week', values = 'rev').reset_index()
info_m = pd.melt(info_m, id_vars = 'id', var_name = 'week', value_name = 'money').sort_values(['id','week'],ascending = [1,0])
info_m = info_m.groupby(['id']).head(3)
info_m = info_m.groupby(['id'])['money'].sum().reset_index()
info_m.head()

,id,money
0,1,466.54
1,2,0.00
2,3,0.00
3,4,0.00
4,5,1812.75


In [12]:
max_date = cnt['week'].max()
max_date

Timestamp('2017-12-25 00:00:00')

In [14]:
info_r = cnt.groupby(['id']).agg({'week': lambda x : (max_date - x.max())}).reset_index().rename(columns = {'week':'recency'})
info_r['recency'] = info_r['recency'].dt.days
info_r.head()

,id,recency
0,1,7
1,2,126
2,3,98
3,4,196
4,5,14


In [17]:
info = pd.merge(pd.merge(info_r, info_f, 'left','id'), info_m, 'left', 'id')
info = info.assign(r = pd.qcut(info['recency'], 5, list('12345')),
                   f = pd.qcut(info['freq'].rank(method = 'first'), 5, list('54321')),
                   m = pd.qcut(info['money'].rank(method = 'first'), 5, list('54321'))
                   )
info.head()

,id,recency,freq,money,r,f,m
0,1,7,2.0,466.54,1,1,2
1,2,126,0.0,0.00,5,5,5
2,3,98,0.0,0.00,4,5,5
3,4,196,0.0,0.00,5,5,5
4,5,14,1.0,1812.75,1,2,1


# Iterators

In [18]:
users = [{'mail': 'gregorythomas@gmail.com', 'name': 'Brett Holland', 'sex': 'M', 'age': 73},
 {'mail': 'hintoncynthia@hotmail.com', 'name': 'Madison Martinez', 'sex': 'F', 'age': 29},
 {'mail': 'wwagner@gmail.com', 'name': 'Michael Jenkins', 'sex': 'M', 'age': 51},
 {'mail': 'daniel79@gmail.com', 'name': 'Karen Rodriguez', 'sex': 'F', 'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]
users

[{'mail': 'gregorythomas@gmail.com',
  'name': 'Brett Holland',
  'sex': 'M',
  'age': 73},
 {'mail': 'hintoncynthia@hotmail.com',
  'name': 'Madison Martinez',
  'sex': 'F',
  'age': 29},
 {'mail': 'wwagner@gmail.com',
  'name': 'Michael Jenkins',
  'sex': 'M',
  'age': 51},
 {'mail': 'daniel79@gmail.com',
  'name': 'Karen Rodriguez',
  'sex': 'F',
  'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]

In [27]:
{k : eval(f'reduce(lambda acc, cur : acc + [cur["{k}"]], users, [])') for k in list(users[0].keys())}

{'mail': ['gregorythomas@gmail.com',
  'hintoncynthia@hotmail.com',
  'wwagner@gmail.com',
  'daniel79@gmail.com',
  'ujackson@gmail.com'],
 'name': ['Brett Holland',
  'Madison Martinez',
  'Michael Jenkins',
  'Karen Rodriguez',
  'Amber Rhodes'],
 'sex': ['M', 'F', 'M', 'F', 'F'],
 'age': [73, 29, 51, 32, 42]}